# Bibliotecas

In [3]:
!pip install  transformers
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00


# Coletando Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [134]:
# Lendo o arquivo csv
reviews_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Scraping reviews.csv', header=None)

# Renomeando as colunas
reviews_df = reviews_df.rename(columns={0: 'ID', 1: 'review', 2: 'rating'})

# Exibindo o dataframe resultante
print(reviews_df)


          ID                                             review  rating
0        NaN                                               text  rating
1        0.0  OK, I fouled up with my Kindle Paperwhite 11th...     4.0
2        1.0  The first thing I thought of when I saw an e-r...     4.0
3        2.0  I don't like upgrades. Every time I'm forced t...     4.0
4        3.0  PROS: Well, I REALLY like the 11th Gen over my...     4.0
...      ...                                                ...     ...
3345  3344.0  I love reading in bed before I go to sleep, I’...     5.0
3346  3345.0           Perfect replacement for an older Kindle.     5.0
3347  3346.0  I can't believe i waited so long to get one of...     5.0
3348  3347.0  If you love to read like myself I recommend yo...     5.0
3349  3348.0  Bought this to replace a stolen Kindle that wa...     5.0

[3350 rows x 3 columns]


In [6]:
reviews_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3350 entries, 0 to 3349
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      3349 non-null   float64
 1   review  3345 non-null   object 
 2   rating  3350 non-null   object 
dtypes: float64(1), object(2)
memory usage: 78.6+ KB


In [135]:
reviews_df = reviews_df.dropna(axis=0)
reviews_df['ID'] = reviews_df['ID'].astype(int)
reviews_df['rating'] = reviews_df['rating'].astype(float)
reviews_df = reviews_df.set_index('ID')

<ipython-input-135-9e42932fd5e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['ID'] = reviews_df['ID'].astype(int)
<ipython-input-135-9e42932fd5e9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['rating'] = reviews_df['rating'].astype(float)


In [8]:
reviews_df.sample(10)

,review,rating
ID,,
2282,The most non intuitive pos reader ever. An idi...,2.0
2456,The screen had a row of dead pixels. For $150 ...,1.0
2124,I am not at all satisfied with this product! I...,5.0
1332,"This is just like new !!!! LOVE, LOVE, this! T...",5.0
2734,Definitely one of the best,1.0
2237,"See video recording, those are two Reoccurring...",1.0
1676,Bought it for my husband. He loves it!,2.0
242,I picked out and received my Kindle Paperwhite...,5.0
2054,Had I known that I was getting into a product ...,2.0


# Pré-Processamento

##Configurar Targets

In [136]:
reviews_df['Sentimento'] = reviews_df['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [10]:
reviews_df

,review,rating,Sentimento
ID,,,
0,"OK, I fouled up with my Kindle Paperwhite 11th...",4.0,0
1,The first thing I thought of when I saw an e-r...,4.0,0
2,I don't like upgrades. Every time I'm forced t...,4.0,0
3,"PROS: Well, I REALLY like the 11th Gen over my...",4.0,0
4,Kindle Signature Paperwhite plus Charging Stan...,4.0,0
...,...,...,...
3344,"I love reading in bed before I go to sleep, I’...",5.0,0
3345,Perfect replacement for an older Kindle.,5.0,0
3346,I can't believe i waited so long to get one of...,5.0,0


In [11]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 0 to 3348
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   review      3344 non-null   object 
 1   rating      3344 non-null   float64
 2   Sentimento  3344 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 104.5+ KB


##Verificando o Balanceamento do Dataset

In [12]:
# contando a quantidade de ocorrências de cada valor na coluna sentimento
counts = reviews_df['Sentimento'].value_counts()

In [13]:
# contando a quantidade de ocorrências de cada valor na coluna sentimento
counts = reviews_df['Sentimento'].value_counts()

# imprimindo os resultados
print(f'Quantidade de Sentimentos Positivos: {counts[0]} - {round(counts[0]/(counts[0]+counts[1])*100,2)}%')
print(f'Quantidade de Sentimentos Negativos: {counts[1]} - {round(counts[1]/(counts[0]+counts[1])*100,2)}%')

Quantidade de Sentimentos Positivos: 2125 - 63.55%
Quantidade de Sentimentos Negativos: 1219 - 36.45%


##Separando dados de testes

In [137]:
from sklearn.model_selection import train_test_split

In [138]:
reviews_df, test_df = train_test_split(reviews_df, test_size=0.25, random_state=42)

In [16]:
len(reviews_df)

2508

##Balanceando as Classes

In [17]:
# separar os exemplos positivos e negativos
negativos = reviews_df[reviews_df['Sentimento'] == 1]
positivos = reviews_df[reviews_df['Sentimento'] == 0]

# reduzir a quantidade de exemplos positivos para o mesmo número de exemplos negativos
positivos_amostrados = positivos.sample(n=len(negativos), random_state=42)

# concatenar os exemplos positivos e negativos amostrados
df_amostrado = pd.concat([negativos, positivos_amostrados], axis=0)

# embaralhar as linhas do DataFrame resultante
df_amostrado = df_amostrado.sample(frac=1, random_state=42).reset_index(drop=True)


In [18]:
counts = df_amostrado['Sentimento'].value_counts()
# imprimindo os resultados
print(f'Quantidade de Sentimentos Positivos: {counts[0]} - {round(counts[0]/(counts[0]+counts[1])*100,2)}%')
print(f'Quantidade de Sentimentos Negativos: {counts[1]} - {round(counts[1]/(counts[0]+counts[1])*100,2)}%')

Quantidade de Sentimentos Positivos: 933 - 50.0%
Quantidade de Sentimentos Negativos: 933 - 50.0%


#Configuração Formato

In [145]:
reviews_df

,review,rating,Sentimento
ID,,,
1389,This kindle is decidedly the best one yet. Wor...,5.0,1
2717,"I only read on my Kindle, but the battery life...",1.0,0
1616,This was my first kindle and I didn’t want to ...,3.0,0
168,I like the non glare screen that I can read ea...,4.0,1
3188,The wife never puts it down,5.0,1
...,...,...,...
1096,This generation of paperwhite is much better t...,5.0,1
1131,I don’t like that the screen is not always off...,5.0,1
1298,I had the previous generation Paperwhite and l...,3.0,0


In [146]:
!pip install datasets
from datasets import Dataset, DatasetDict
df = reviews_df.drop('rating', axis=1)
df.columns = ['text', 'label']
df = df.set_index('text')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [147]:
# criar um dicionário mapeando nomes de conjuntos de dados para partes do dataframe
data_dict = {'train': df.iloc[:2000], 'test': df.iloc[2000:], 'unsupervised': df}

# criar objetos Dataset para cada conjunto de dados
datasets = {k: Dataset.from_pandas(v) for k, v in data_dict.items()}

# criar um objeto DatasetDict a partir dos objetos Dataset criados
dataset_dict = DatasetDict(datasets)

# imprimir informações sobre o objeto DatasetDict criado
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 508
    })
    unsupervised: Dataset({
        features: ['label', 'text'],
        num_rows: 2508
    })
})


#Baixando o Modelo Pre-treinado

In [201]:
base_model = TFAutoModel.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertModel: ['classifier', 'dropout_37']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [149]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [150]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=200)

In [151]:
emotions_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [152]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 508
    })
    unsupervised: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [153]:
emotions_encoded.set_format('tf',columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

In [154]:
# setting BATCH_SIZE to 12.
BATCH_SIZE = 12

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [155]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(12, 200), dtype=int64, numpy=
array([[  101,   151, 35925, ...,     0,     0,     0],
       [  101, 11526, 10855, ...,     0,     0,     0],
       [  101, 10372, 16540, ...,     0,     0,     0],
       ...,
       [  101, 10372, 10127, ..., 10336, 18092,   102],
       [  101, 11838, 10114, ...,     0,     0,     0],
       [  101, 31377, 15724, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(12, 200), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(12, 200), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([0 1 1 0 1 1 1 1 0 1 1 1]

#Definição, treinamento e parametros

In [194]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model):
        super().__init__()
        self.bert = bert_model
        self.fc1 = tf.keras.layers.Dense(32, activation='relu')
        self.fc2 = tf.keras.layers.Dense(1, activation='sigmoid')
        
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        x = self.fc1(x)
        return self.fc2(x)

In [202]:
base_model.trainable = False
classifier = BERTForClassification(base_model)
classifier.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy()])

In [203]:
history = classifier.fit(
    train_dataset,
    epochs=5, validation_data=test_dataset)

Epoch 1/5
70/70 [==============================] - 37s 340ms/step - loss: 0.6237 - binary_accuracy: 0.6603 - val_loss: 0.6105 - val_binary_accuracy: 0.6654
Epoch 2/5
70/70 [==============================] - 21s 297ms/step - loss: 0.5952 - binary_accuracy: 0.7165 - val_loss: 0.6151 - val_binary_accuracy: 0.6614
Epoch 3/5
70/70 [==============================] - 21s 297ms/step - loss: 0.5883 - binary_accuracy: 0.6986 - val_loss: 0.6207 - val_binary_accuracy: 0.6614
Epoch 4/5
70/70 [==============================] - 21s 296ms/step - loss: 0.5866 - binary_accuracy: 0.7045 - val_loss: 0.7104 - val_binary_accuracy: 0.6476
Epoch 5/5
70/70 [==============================] - 21s 296ms/step - loss: 0.5826 - binary_accuracy: 0.7153 - val_loss: 0.6194 - val_binary_accuracy: 0.6535


In [197]:
# Unfreeze the base model
base_model.trainable = True
classifier = BERTForClassification(base_model)

classifier.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy()])

history = classifier.fit(
    train_dataset,
    epochs=3, validation_data=test_dataset)

Epoch 1/3
70/70 [==============================] - 84s 665ms/step - loss: 0.3662 - binary_accuracy: 0.8636 - val_loss: 0.7533 - val_binary_accuracy: 0.7165
Epoch 2/3
70/70 [==============================] - 43s 611ms/step - loss: 0.2419 - binary_accuracy: 0.9127 - val_loss: 0.7316 - val_binary_accuracy: 0.6969
Epoch 3/3
70/70 [==============================] - 43s 614ms/step - loss: 0.1883 - binary_accuracy: 0.9306 - val_loss: 0.8810 - val_binary_accuracy: 0.6594


In [204]:
df = test_df.drop('rating', axis=1)
df.columns = ['text', 'label']
df = df.set_index('text')

# criar um dicionário mapeando nomes de conjuntos de dados para partes do dataframe
data_dict = {'train': df.iloc[:len(df)], 'test': df.iloc[len(df)-1:], 'unsupervised': df}

# criar objetos Dataset para cada conjunto de dados
datasets = {k: Dataset.from_pandas(v) for k, v in data_dict.items()}

# criar um objeto DatasetDict a partir dos objetos Dataset criados
dataset_dict = DatasetDict(datasets)

emotions_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

emotions_encoded.set_format('tf',columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

inp, out = next(iter(train_dataset))

y_pred = classifier.predict(train_dataset)


Map:   0%|          | 0/836 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/836 [00:00<?, ? examples/s]

70/70 [==============================] - 15s 173ms/step


#Resultado

In [205]:
 # Arredonda cada elemento da lista e converte em inteiro
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
y = test_df['Sentimento'].values
y = np.array(y, dtype=np.int32)
y_pred =  np.round(y_pred).astype(int)
accuracy= accuracy_score(y_pred,  y)*100
f1_Score = f1_score(y_pred,  y)*100
print(f'Bert Accuracy Score -> {accuracy}\nBert F1 Score-> {f1_Score}')


Bert Accuracy Score -> 64.11483253588517
Bert F1 Score-> 77.81065088757396


# Inspecionando os Resultados

In [206]:
# Encontra os índices dos 5 primeiras avaliações negativas
n= 5
indices = [i for i in range(len(y_pred)) if y_pred[i] == 0][:n]
print(f'As {n} primeiras avaliações que o modelo considerou como negativas:\n\n')
for i in indices:

  print(f'Avaliação: {test_df.review.iloc[i]}  - Rating {test_df.rating.iloc[i]} \n')

As 5 primeiras avaliações que o modelo considerou como negativas:


Avaliação: Unfortunately screen froze after daily use about 11 months after it was gifted as a Christmas present. Unfortunately it was purchased around Cyber Monday so warranty expired 2 days before it broke and could not be reset or used. Amazon customer service only able to give a 15% discount on new device given out of warranty period. Poor durability and poor customer service  - Rating 5.0 

Avaliação: LOVE LOVE LOVE this Kindle! Perfect size, amazing battery life, and beautiful screen  - Rating 4.0 

Avaliação: I had great experience with 6 Amazon Kindles in the past
so I bought the latest Paperwhite for my son as a gift.
- The 6.8'' screen is noticeable larger than previous gens and easier to read
- Type-C charging port is an excellent upgrade so we no longer to carry a micro-usb cable
- Warm back light reduces eye strain in the evening
- Fast charging speed at around 8 watts (5v x 1.2a)
- Still very lightweight
